In [ ]:
!python -m pip install --upgrade pip
!python -m pip install autogluon

In [3]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [4]:
data_url = 'df_'
train_data = TabularDataset(f'{data_url}train.csv')
train_data.head()

,Dst Port,Protocol,Fwd Pkt Len Std,Bwd Pkt Len Min,Flow Byts/s,Fwd IAT Mean,Bwd IAT Tot,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,...,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Std,Active Max,Active Min,Idle Max,Idle Min,Label
0,0.506211,0.352941,0.026531,0.0,3.702185e-07,0.898484,0.000149,0.000113,0.000131,0.000018,...,0.003632,0.003235,0.000004,0.666667,0.0,0.0,0.0,0.0,0.0,Benign
1,0.006760,0.352941,0.000000,0.0,0.000000e+00,0.897540,0.000000,0.000000,0.000000,0.000000,...,0.979767,0.000000,0.000000,0.416667,0.0,0.0,0.0,0.0,0.0,Benign
2,0.123295,0.352941,0.000000,0.0,0.000000e+00,0.897541,0.000000,0.000000,0.000000,0.000000,...,0.003922,0.000000,0.000000,0.416667,0.0,0.0,0.0,0.0,0.0,Botnet
3,0.000320,0.352941,0.000000,0.0,0.000000e+00,0.897540,0.000000,0.000000,0.000000,0.000000,...,0.410217,0.000015,0.000000,0.833333,0.0,0.0,0.0,0.0,0.0,Brute-force
4,0.000320,0.352941,0.000000,0.0,0.000000e+00,0.897540,0.000000,0.000000,0.000000,0.000000,...,0.410217,0.000015,0.000000,0.833333,0.0,0.0,0.0,0.0,0.0,Brute-force


In [5]:
label = 'Label'
train_data[label].describe()

count     3623992
unique          7
top        Benign
freq      1811996
Name: Label, dtype: object

In [6]:
save_path = 'agModels-predictClass-deployment/'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "/agModels-predictClass-deployment/"
=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.9.7
Ope

	Not enough memory to train ExtraTreesEntr... Skipping this model.
Fitting model: XGBoost ...
	0.9557	 = Validation score   (accuracy)
	712.3s	 = Training   runtime
	0.93s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.9536	 = Validation score   (accuracy)
	1539.26s	 = Training   runtime
	0.16s	 = Validation runtime
Fitting model: LightGBMLarge ...
	0.9544	 = Validation score   (accuracy)
	83.97s	 = Training   runtime
	0.19s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'XGBoost': 0.75, 'CatBoost': 0.143, 'LightGBMXT': 0.036, 'LightGBM': 0.036, 'LightGBMLarge': 0.036}
	0.9558	 = Validation score   (accuracy)
	4.47s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 5333.34s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/agModels-predictClass-deployment/")


In [7]:
test_data = TabularDataset(f'{data_url}test.csv')

y_pred = predictor.predict(test_data.drop(columns=[label]))
y_pred.head()

Loaded data from: df_test.csv | Columns = 35 / 35 | Rows = 905998 -> 905998


0         Benign
1         Benign
2    DDoS attack
3         Botnet
4         Benign
Name: Label, dtype: object

In [8]:
predictor.evaluate(test_data, silent=True)

{'accuracy': 0.9550639184633961,
 'balanced_accuracy': 0.847015399038689,
 'mcc': 0.9341109785145134}

In [9]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.955064,0.955850,accuracy,41.597914,1.608641,2021.902024,0.225224,0.003000,4.470335,2,True,10
1,XGBoost,0.954977,0.955712,accuracy,25.941797,0.934252,712.295400,25.941797,0.934252,712.295400,1,True,7
2,CatBoost,0.954449,0.955132,accuracy,0.577335,0.028996,1031.796235,0.577335,0.028996,1031.796235,1,True,6
3,LightGBMLarge,0.954165,0.954360,accuracy,5.440263,0.192000,83.967020,5.440263,0.192000,83.967020,1,True,9
4,LightGBM,0.953758,0.953808,accuracy,2.411585,0.112767,64.416176,2.411585,0.112767,64.416176,1,True,5
5,NeuralNetTorch,0.953331,0.953560,accuracy,4.415244,0.163000,1539.257509,4.415244,0.163000,1539.257509,1,True,8
6,NeuralNetFastAI,0.952318,0.952621,accuracy,5.326926,0.168329,1535.297565,5.326926,0.168329,1535.297565,1,True,3
7,KNeighborsUnif,0.951580,0.951959,accuracy,2308.037873,97.933544,11.491163,2308.037873,97.933544,11.491163,1,True,1
8,LightGBMXT,0.951100,0.951876,accuracy,7.001711,0.337626,124.956860,7.001711,0.337626,124.956860,1,True,4
9,KNeighborsDist,0.948034,0.948731,accuracy,2417.098345,98.094985,2.393082,2417.098345,98.094985,2.393082,1,True,2
